In [ ]:
!unzip "/content/drive/MyDrive/deepfake.zip"

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/Validation/Real/real_5499.jpg  
  inflating: Dataset/Validation/Real/real_55.jpg  
  inflating: Dataset/Validation/Real/real_550.jpg  
  inflating: Dataset/Validation/Real/real_5500.jpg  
  inflating: Dataset/Validation/Real/real_5501.jpg  
  inflating: Dataset/Validation/Real/real_5502.jpg  
  inflating: Dataset/Validation/Real/real_5503.jpg  
  inflating: Dataset/Validation/Real/real_5504.jpg  
  inflating: Dataset/Validation/Real/real_5505.jpg  
  inflating: Dataset/Validation/Real/real_5506.jpg  
  inflating: Dataset/Validation/Real/real_5507.jpg  
  inflating: Dataset/Validation/Real/real_5508.jpg  
  inflating: Dataset/Validation/Real/real_5509.jpg  
  inflating: Dataset/Validation/Real/real_551.jpg  
  inflating: Dataset/Validation/Real/real_5510.jpg  
  inflating: Dataset/Validation/Real/real_5511.jpg  
  inflating: Dataset/Validation/Real/real_5512.jpg  
  inflating: Dataset/Validation/Real/real_5513.jpg  

In [ ]:
import matplotlib.pyplot as plt
# import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential


In [ ]:
train_dir = '/content/Dataset/Train'
test_dir = '/content/Dataset/Test'
validation_dir = '/content/Dataset/Validation'

In [ ]:
def normalize_image(image, labels):
  image = tf.cast(image, tf.float32) / 255.0
  return image, labels

In [ ]:
IMG_SIZE = (256, 256)

train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode = 'categorical',
                                                                 batch_size = 32,
                                                                 image_size= IMG_SIZE)

validation_data = tf.keras.preprocessing.image_dataset_from_directory(validation_dir,
                                                                 label_mode = 'categorical',
                                                                 batch_size = 32,
                                                                 image_size= IMG_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                 label_mode = 'categorical',
                                                                 batch_size = 32,
                                                                 image_size= IMG_SIZE,
                                                                shuffle = False)

Found 140002 files belonging to 2 classes.
Found 39428 files belonging to 2 classes.
Found 10905 files belonging to 2 classes.


In [ ]:
train_data = train_data.map(normalize_image)
validation_data = validation_data.map(normalize_image)
test_data = test_data.map(normalize_image)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense

model_CNN = Sequential([

    # First Conv2D block
    Conv2D(filters=8, kernel_size=5, input_shape=(256, 256, 3), activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),

    # Second Conv2D block
    Conv2D(filters=16, kernel_size=4, activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),
    Dropout(0.2),

    # Third Conv2D block
    Conv2D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),

    # Fourth Conv2D block
    Conv2D(filters=64, kernel_size=2, activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),

    # Fifth Conv2D block
    Conv2D(filters=128, kernel_size=1, activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),
    Dropout(0.2),

    # Fully connected layers
    Flatten(),
    Dropout(0.3),
    Dense(units=64, activation='softmax'),
    Dense(units=20, activation='softmax'),
    Dense(units=2, activation='softmax')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# # instantiating the model in the strategy scope creates the model on the TPU
# with tpu_strategy.scope():
model_CNN.compile(optimizer = tf.keras.optimizers.Adam(),
                loss = 'BinaryCrossentropy',
                metrics=['accuracy'])

In [ ]:
hist = model_CNN.fit(train_data,
                    epochs = 10,
                    validation_data = validation_data,
                    validation_steps = int(0.5 * len(validation_data))
                    )

Epoch 1/10
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 183s 38ms/step - accuracy: 0.6883 - loss: 0.6179 - val_accuracy: 0.7186 - val_loss: 0.5473
Epoch 2/10
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 177s 35ms/step - accuracy: 0.7625 - loss: 0.4934 - val_accuracy: 0.7557 - val_loss: 0.4950
Epoch 3/10
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 186s 31ms/step - accuracy: 0.7820 - loss: 0.4431 - val_accuracy: 0.0000e+00 - val_loss: 1.0924
Epoch 4/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


4376/4376 ━━━━━━━━━━━━━━━━━━━━ 159s 35ms/step - accuracy: 0.8141 - loss: 0.4002 - val_accuracy: 0.7934 - val_loss: 0.4441
Epoch 5/10
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 197s 34ms/step - accuracy: 0.8442 - loss: 0.3481 - val_accuracy: 0.8207 - val_loss: 0.4022
Epoch 6/10
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 188s 31ms/step - accuracy: 0.8599 - loss: 0.3145 - val_accuracy: 0.5000 - val_loss: 0.4232
Epoch 7/10
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 150s 34ms/step - accuracy: 0.8740 - loss: 0.2890 - val_accuracy: 0.8274 - val_loss: 0.3702
Epoch 8/10
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 147s 33ms/step - accuracy: 0.8848 - loss: 0.2774 - val_accuracy: 0.8536 - val_loss: 0.3533
Epoch 9/10
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 191s 31ms/step - accuracy: 0.8952 - loss: 0.2586 - val_accuracy: 1.0000 - val_loss: 0.1680
Epoch 10/10
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 147s 34ms/step - accuracy: 0.8961 - loss: 0.2528 - val_accuracy: 0.8661 - val_loss: 0.3280


In [ ]:
model_CNN.evaluate(test_data)

341/341 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - accuracy: 0.7626 - loss: 0.5228


[0.4172833263874054, 0.822650134563446]

In [ ]:
# Save the trained model to a file
model_CNN.save('model_CNN.h5')
print("Model saved successfully.")

Model saved successfully.
